## Detection of sentence endings in run-on sentences

In [1]:
import numpy as np
import random
from tqdm import tqdm
import json
from nltk.tokenize import sent_tokenize
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression

In [2]:
import spacy
nlp = spacy.load('en_core_web_md')

Для завдання визначення слів, що закінчують речення в "run-on sentences", я спробував пошукати дані серед датасетів Kaggle. Дані з Твіттера відразу показали низький результат, і вони в будь-якому разі погано підходять - у Твіттері багато специфічних символів і скорочень, може оминатись пунктуація тощо. Це ускладнює роботу для парсера і водночас не дуже схоже на реальні run-on sentences.

Тексти з блогів підійшли краще, але зрештою я зупинився на транскриптах із TED Talks (https://www.kaggle.com/rounakbanik/ted-talks). Спічі на TED мають одночасно ознаки розмовного і наукового стилів. Речення зазвичай короткі, а саме поєднання коротких речень, як мені здається, найчастіше призводить до "забування" про крапки між реченнями.

2467 транскриптів містять 277465 речень. Для тренування класифікатора я "склеїв" послідовні речення купками по 3-5 (спочатку я робив по 1-3, як у тестувальних даних, але виявилось, що класифікатор трохи краще тренується на більших склеєних реченнях). Склеєні, токенізовані речення з лейблом True/False для кожного слова я зберіг у json, щоб тренувальні дані були тієї ж форми, що і тестувальні. Функції, які обробляють і зберігають тренувальні дані, знаходяться у файлі `prepare_data.py`.

Завантажимо тестові дані і подивимось, скільки в ньому закінчень речень.

In [3]:
test_set = json.load(open('run-on-test.json'))

from collections import Counter
counts = []
for d in test_set:
    true_count = 0
    for w in d:
        if w[1] == True:
            true_count += 1
    counts.append(true_count)
    
Counter(counts).most_common()

[(1, 145), (0, 50), (2, 5)]

У 50 випадках речення взагалі не склеєні, у 145 - склеєно два речення, у п'яти випадках - три.

Оскільки тренувальні та тестувальні речення заздалегідь токенізовані, потрібна функція, яка робитиме spacy Doc із токенів (це потребує, крім самих токенів, визначення наявності пробілів після них).

In [4]:
def make_spaceable(token_list):
    """
    Make possible use of spacy doc
    with already tokenized text
    """
    tokens = [t[0] for t in token_list]
    spaces = []
    for i, token in enumerate(tokens):
        if i == len(tokens)-1:
            spaces.append(False)
        elif ((tokens[i+1] in '!%),.:;?}')
              or (tokens[i+1].startswith("'") and len(tokens[i+1]) > 1)):
            spaces.append(False)
        else:
            spaces.append(True)
    assert len(spaces) == len(tokens)
    doc = spacy.tokens.doc.Doc(
        nlp.vocab, words=tokens, spaces=spaces)
    for name, proc in nlp.pipeline:
        doc = proc(doc)        
    return doc

Завантажуємо тренувальні дані.

In [5]:
PATH = '/mnt/hdd/Data/NLP/'
ted_set = json.load(open(PATH+'ted_data.json'))
print(len(ted_set))
print(sum(len(sent) for sent in ted_set))

70273
5795006


У нас вийшло близько 70 тисяч склеєних речень з майже 5,8 мільйонів токенів. На практиці виявилось, що поєднання такої кількості фіч з такою кількістю токенів якраз майже цілком заповнює мої 16 гб оперативної пам'яті.

Функція, яка дістає фічі з кожного слова токенізованого склеєного речення. Окрім самого слова, функція дивиться на контекст - два слова ліворуч і два слова праворуч. Таким чином функція аналізує, крім уніграмів, біграми і триграми.

Особливості слів, які мені здались корисними для визначення, чи є слово закінченням речення:
- саме слово, його лемма і написання з малої літери;
- чи є воно з великої літери;
- його довжина, його положення в речення (кількість символів від початку речення до слова);
- POS-тег, тип залежності;
- чи є пунктуацією;
- усе те саме для одного або двох правих і лівих слів (те, що здалось важливішим, для двох правих і лівих);
- ліва та права біграми, якщо можливо;
- кількість лівих та правих "дітей" слова у синтаксичному дереві (для закінчення речення, правих дітей, по ідеї, не повинно бути);
- найнижчий спільний предок слова та двох правих і двох лівих слів у синтаксичному дереві: у випадку закінчення речення, спільних предків з правими словами бути не повинно або вони будуть дуже високо.

Для отримання всієї інформації про токени я використав spacy, модель англійської мови середнього розміру (велика модель не давала відчутного покращення).

In [6]:
def get_features(token_data):
    """
    Make a list of features and two lists of labels
    from our data
    """
    data, labels = [], []
    wrong_count = 0
    for token_list in tqdm(token_data):
        doc = make_spaceable(token_list)
        sent_len = len(doc)
        lca_matrix = doc.get_lca_matrix()
        if not sent_len == len(token_list):
            continue
        for token in doc:
            i = token.i
            features = {
                'word': token.text,
                'word_lower': token.lower_,
                'word_lemma': token.lemma_,
                'word_pos': token.pos_,
                'word_dep': token.dep_,
                'word_capitalized': True if token.is_title else False,
                'word_ispunct': token.is_punct,
                'word_n_lefts': token.n_lefts,
                'word_n_rights': token.n_rights,
                'word_idx': token.idx,
                'length': len(token.text)
            }
            if i > 0:
                features.update({
                    'word-1': doc[i-1].lower_,
                    'w-1pos': doc[i-1].pos_,
                    'w-1dep': doc[i-1].dep_,
                    'w-1_lca': lca_matrix[i, i-1],
                    'word-1_capitalized': True if doc[i-1].is_title else False,
                    'word-1_ispunct': True if doc[-1].is_punct else False
                })
            if i > 1:
                features.update({
                    'word-2': doc[i-2].lower_,
                    'w-2dep': doc[i-2].dep_,
                    'w-2_lca': lca_matrix[i, i-2],
                    'left_bigram': doc[i-2].lower_ + '_' + doc[i-1].lower_
                })
            if i < sent_len - 1:
                lca = lca_matrix[i, i+1]
                features.update({
                    'word+1': doc[i+1].lower_,
                    'w+1pos': doc[i+1].pos_,
                    'w+1dep': doc[i+1].dep_,
                    'w+1_lca': lca if lca > -1 else 100,
                    'word+1_capitalized': True if doc[i+1].is_title else False,
                    'word+1_ispunct': True if doc[i+1].is_punct else False,
                    'word+1_n_rights': doc[i+1].n_rights,
                    'word+1_n_lefts': doc[i+1].n_lefts
                })
            if i < sent_len - 2:
                lca = lca_matrix[i, i+2]
                features.update({
                    'word+2': doc[i+2].lower_,
                    'w+2dep': doc[i+2].dep_,
                    'w+2_lca': lca if lca > -1 else 100,
                    'right_bigram': (doc[i+1].lower_ + '_' + doc[i+2].lower_)
                })
            last_lab = token_list[i][1]
            labels.append(last_lab)
            data.append(features)
    return data, labels

Отримаємо наші фічі, для тренувального і для тестувального датасету.

In [7]:
data, labels = get_features(ted_set)

100%|██████████| 70273/70273 [48:08<00:00, 24.33it/s]


In [8]:
# probably it can free some memory, I'm not sure
del ted_set

Тестових сетів у нас два, тому валідаційний, "зовнішній" тестсет буде позначатись як test_test

In [9]:
test_test_data, test_test_labels = get_features(test_set)

100%|██████████| 200/200 [00:02<00:00, 87.46it/s]


In [10]:
from sklearn.model_selection import train_test_split
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2,
                                                                   random_state=505)

In [11]:
from sklearn.feature_extraction import DictVectorizer
vec = DictVectorizer()
train_features = vec.fit_transform(train_data)
test_features = vec.transform(test_data)

Для класифікатора я використав логістичну регресію із пакету sklearn. Параметр penalty='l1' на цих даних працює краще, ніж 'l2', в усьому іншомі параметри дефолтні. Параметр class_weight='balanced' не дає покращення. Класифікатор LogisticRegressionCV (поєднання класифікатора і кросвалідації) я теж пробував, але він на диво дав трохи гірший результат.

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(penalty='l1', random_state=505)
clf.fit(X=train_features, y=train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=505, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Результати на тестових даних із транскриптів TED:

In [13]:
pred1 = clf.predict(test_features)
print(classification_report(test_labels, pred1, digits=3))

             precision    recall  f1-score   support

      False      0.991     0.996     0.993   1117930
       True      0.871     0.745     0.803     41072

avg / total      0.986     0.987     0.987   1159002



Результати на власне тестовому, валідаційному датасеті:

In [14]:
test_test_features = vec.transform(test_test_data)
pred2 = clf.predict(test_test_features)
print(classification_report(test_test_labels, pred2, digits=3))

             precision    recall  f1-score   support

      False      0.992     0.990     0.991      4542
       True      0.726     0.768     0.746       155

avg / total      0.983     0.983     0.983      4697



Звичайно, в усіх результатах нас найбільше цікавить не середній F1 (завжди близький до одиниці), а F1 для лейблу "True", а також precision i recall для цього ж лейблу. 

На жаль, F1 вище ніж 0.74 так і не вдалось досягнути. Могло б допомогти більше часу або оперативної пам'яті, але навряд чи з цими даними та умовами вдалось би витягнути на більш ніж 0.75-0.76.

Цікаво, що на тестовому сеті з транскриптів TED precision вищий за recall, а у валідаційному сеті навпаки.

Інші підходи, які я пробував:

- даунсемплінг для лейблів False та апсемплінг для лейблів True, з допомогою функці sklearn.utils.resample: обидва підходи дають незначне погіршення на тестовому сеті з транскриптів та значне погіршення на валідаційному сеті - важко сказати, чому;

- sklearn CRF suite - CRF для текстової класифікації. Працює непогано, але f1 стабільно виходив на 0.01 гірший, ніж з використанням просто логістичної регресії;

- sklearn SGDClassifier (loss='hinge', що означає використання лінійної SVM) - досить помітне погіршення результату на обох тестових вибірках;

- зменшення кількості фіч (якщо видалити декілька з тих, які здаються менш важливими) дуже-дуже незначно погіршує результат, тому з точки зору швидкості розрахунків є сенс використати менше фіч. Але для максимально можливого результату тут я їх не видаляв.

Частково ці підходи можна подивитись в іншому .ipynb файлі `Draft`.

В цілому в усіх випадках підхід, який працює гірше для тестової вибірки з транскриптів, так само гірше працює і для валідаційної вибірки, але інколи різниця між показниками для двох вибірок значно більша, ніж для логістичної регресії.

Вже готуючись здавати домашнє, я розумію, що можна було спробувати краще зробити фічі про найближчого спільного предка і більше попрацювати на рівні біграм та триграм (я так і не дійшов до біграм з Гугла). Але для початку і це, можливо, нормальний результат.

<hr>

P.S. Вже після дедлайну, я ще кілька разів прогнав логістичну регресію на вибірці з цих даних. Навіть вибірки з 5000 речень і фіч ЛИШЕ для поточного слова і наступного слова достатньо, щоб отримати F1 0.67 для валідаційних даних. Причому якщо брати фічі тільки для поточного слова, без жодного контексту, F1 близький до нуля. Оскільки у випадку останнього слова у реченні наступне слово має низку особливостей - це завжди не пунктуаційний знак, часто пишеться з великої літери, і парсер може зрозуміти, що це слово має інший рут - то логічно, що саме фічі наступного слова дають найбільший приріст до результату.